In [1]:
from copy import deepcopy
from dataclasses import dataclass
import multiprocessing as mp
from re import T
import torchvision.models as models
import torch
import copy
import torch.nn.functional as F
#from model import net
import random
import multiprocessing as mp
import os
import os
import torch
import torch.distributed as dist
import torch.multiprocessing as mp
from torchvision import datasets,transforms
import random
from torch import optim,ceil
import torch.nn.functional as F
from torch import nn
import pickle
import numpy as np
import os
import pandas as pd
from torchvision.io import read_image
from torch.utils.data import Dataset,DataLoader
from torchvision import transforms
import torch
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, classification_report, confusion_matrix


In [2]:

from torch.utils.data import Dataset,DataLoader
import numpy as np

class plantdisease(Dataset):
    def __init__(self, df,
                  img_dir='data/oneFolder'
                 , isTest=False):
        self.img_labels=df
        self.img_dir='../../Data/DataResized512/'
       # self.train, self.test = train_test_split(self.img_labels, test_size=0.05)

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.img_labels['image'][idx])

            image = (read_image(img_path).float())
            self.transform=transforms.Compose([transforms.Resize((342,512)),
        transforms.Normalize((124.4455, 159.9584, 104.1832), (47.1528, 41.4626, 49.2424))])
            image = self.transform(image)
            label = self.img_labels['labels'][idx]
            label=np.fromstring(label[1:-1], dtype=np.int,sep=',')
            return image, label

In [3]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	torch.cuda.empty_cache()
	print("Total size:", total_size)


In [4]:
df=pd.read_csv('../../Data/data10k.csv')
df=df.drop('Unnamed: 0',axis=1)
train,valid_test=train_test_split(df,test_size=0.2)
valid,test=train_test_split(valid_test,test_size=0.5)

train_data=plantdisease(df=train.reset_index())
test_data=plantdisease(df=test.reset_index())
valid_data=plantdisease(df=valid.reset_index())

model=models.mobilenet_v2(pretrained=True)
model.classifier[1]=(torch.nn.Linear(in_features=1280, out_features=6, bias=True))

In [5]:
from GPUtil import showUtilization as gpu_usage
from numba import cuda

def free_gpu_cache():
    print("Initial GPU Usage")
    gpu_usage()                             

    torch.cuda.empty_cache()

    cuda.select_device(0)
    cuda.close()
    cuda.select_device(0)

    print("GPU Usage after emptying the cache")
    gpu_usage()


    dump_tensors()


In [6]:

def confusion_matrixCal(target,pred):
  matrix=0
  for i in range(0,target.shape[0]):
    matrix+=confusion_matrix(target[i], pred[i],labels=[1, 0])
  return matrix

#Test train function

config = {
        "lr": 0.0001,
        "batch_size": 6
        ,"epoch":20
    }
def client_trainGPU(config):

  device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
  optimizer =optim.Adam(model.parameters(),lr=config["lr"],weight_decay=9e-05)

  train_dataloader=DataLoader(train_data,batch_size=int(config["batch_size"]))
  test_dataloader=DataLoader(test_data,batch_size=int(config["batch_size"]))
  valid_dataloader=DataLoader(valid_data,batch_size=int(config["batch_size"]))

  lossHist=[]
  lossHist_test=[]


  for epoc in range(config['epoch']):
      model.to(device)
      #
      total=0
#
      correct=0
      train_loss = 0.0
      confusionMatrix=0
      accuracyHist=[]
      train_step=0
      print('start Epoch ',epoc)
      for i, datas in enumerate(((train_dataloader)),0):
          data, target=datas
          data,target=data.to(device),target.to(device)
          
          
          output = model(data)

          loss = torch.nn.BCEWithLogitsLoss()(output, target.float())

          train_loss += loss.item()
          train_step+=1


          ouput=torch.sigmoid(output)
          predicted = torch.round(ouput)
          target=target.cpu().detach()
          data=data.cpu().detach()
          predicted=predicted.cpu().detach()
          accuracyHist.append(accuracy_score(target,predicted))
          confusionMatrix+=confusion_matrixCal(target,predicted)
          optimizer.zero_grad(set_to_none=True)
          loss.backward()
          optimizer.step()
      print('Client ID ',  ', epoch ',
        epoc, ': ', train_loss / int(config["batch_size"]))
      lossHist.append(train_loss / int(config["batch_size"]))
      print('accuracy {}'.format(np.mean(accuracyHist)))  

      #Evaluation

      ######
      confusionMatrix_test=0
      accuracyHist_test=[]
      test_loss = 0.0
      test_step=0
      model.cpu()
      model.eval()
      with torch.no_grad():
        for i, datas in enumerate(((test_dataloader)),0):
            data, target=datas
            
            output = model(data)
            
            loss = torch.nn.BCEWithLogitsLoss()(output, target.float())
            ouput=torch.sigmoid(output)
            predicted = np.round(ouput)
            total += target.size(0)
            test_loss += loss.item()
            test_step+=1

            accuracyHist_test.append(accuracy_score(target,predicted))
            confusionMatrix_test+=confusion_matrixCal(target,predicted)
      print('test_loss {}'.format(test_loss/test_step))
      lossHist_test.append(test_loss/test_step)
      print('accuracy {}'.format(np.mean(accuracyHist_test)))
      train_loss=train_loss / train_step
      test_loss=test_loss/test_step

      TP_test=confusionMatrix_test[0][0]
      FP_test=confusionMatrix_test[0][1]
      FN_test=confusionMatrix_test[1][0]
      TN_test=confusionMatrix_test[1][1]
      accuracy_test=(TP_test+TN_test)/(FP_test+FN_test+TP_test+TN_test)
      recal_test=TP_test/(TP_test+FN_test)
      precision_test=TP_test/(TP_test+FP_test)
      negativePredictiveValue_test=TN_test/(TN_test+FN_test)
      missRate_test=FN_test/(TP_test+FN_test)
      fallOut_test=FP_test/(TN_test+FP_test)
      f1_test=(2*precision_test*recal_test)/(precision_test+recal_test)


      TP_train=confusionMatrix[0][0]
      FP_train=confusionMatrix[0][1]
      FN_train=confusionMatrix[1][0]
      TN_train=confusionMatrix[1][1]
      accuracy_train=(TP_train+TN_train)/(FP_train+FN_train+TP_train+TN_train)
      recal_train=TP_train/(TP_train+FN_train)
      precision_train=TP_train/(TP_train+FP_train)
      negativePredictiveValue_train=TN_train/(TN_train+FN_train)
      missRate_train=FN_train/(TP_train+FN_train)
      fallOut_train=FP_train/(TN_train+FP_train)
      f1_train=(2*precision_train*recal_train)/(precision_train+recal_train)

      wandb.log({
            'epoc': epoc, 
            'train_loss': train_loss,
            'val_loss': test_loss,

            
            'TP_test':TP_test,
            'FP_test':FP_test,
            'FN_test':FN_test,
            'TN_test':TN_test,
            'accuracy_test':accuracy_test,
            'recal_test':recal_test,
            'precision_test':precision_test,
            'negativePredictiveValue_test':negativePredictiveValue_test,
            'f1_test':f1_test,


            'TP':TP_train,
            'FP':FP_train,
            'FN':FN_train,
            'TN':TN_train,
            'accuracy_train':accuracy_train,
            'recal_train':recal_train,
            'precision_train':precision_train,
            'negativePredictiveValue_train':negativePredictiveValue_train,
            'f1_train':f1_train,
          })

  free_gpu_cache()
  print("Finished Training")



In [7]:
import wandb
import numpy as np 
import random

# Define sweep config
sweep_configuration = {
    'method': 'random',
    'name': 'sweep',
    'metric': {'goal': 'maximize', 'name': 'val_acc'},
    'parameters': 
    {
        'batch_size': {'values': [8,16]},
        'epochs': {'values': [5]},
        'lr': {'max': 0.0002, 'min': 0.00001}
     }
}
sweep_id = wandb.sweep(sweep=sweep_configuration, project='modeltunning_16')

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 0metq8be
Sweep URL: https://wandb.ai/makkuror/modeltunning_16/sweeps/0metq8be


In [8]:
os.environ["WANDB_API_KEY"] = '831281456cbf2d04f75207adc2e9811dc4fd32cf'
def main():
    dump_tensors()

    run = wandb.init()
    
    # note that we define values from `wandb.config` instead 
    # of defining hard values

    config = {
        "lr": wandb.config.lr,
        "batch_size":wandb.config.batch_size,
        "epoch":wandb.config.epochs
    }
    import warnings 
    warnings.filterwarnings('ignore')
    a=client_trainGPU(config)



# Start sweep job.
wandb.agent(sweep_id, function=main, count=20)

wandb: Agent Starting Run: 5yzdd0zf with config:
wandb: 	batch_size: 8
wandb: 	epochs: 5
wandb: 	lr: 5.7166084355680035e-05
e:\Anaconda\envs\pytorchGPUI\lib\site-packages\torch\distributed\distributed_c10d.py:181: UserWarning: torch.distributed.reduce_op is deprecated, please use torch.distributed.ReduceOp instead
  warnings.warn(
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Total size: 0


wandb: Currently logged in as: makkuror. Use `wandb login --relogin` to force relogin


start Epoch  0
Client ID  , epoch  0 :  27.24395285313949
accuracy 0.6365
test_loss 0.12094356102496386
accuracy 0.806
start Epoch  1
Client ID  , epoch  1 :  17.1249247577216
accuracy 0.785875
test_loss 0.1400116395931691
accuracy 0.788
start Epoch  2
Client ID  , epoch  2 :  13.200377291825134
accuracy 0.83525
test_loss 0.13196187509596347
accuracy 0.805
start Epoch  3
Client ID  , epoch  3 :  11.186544140378828
accuracy 0.85775
test_loss 0.12909394036978483
accuracy 0.817
start Epoch  4
Client ID  , epoch  4 :  9.933420970788575
accuracy 0.86725
test_loss 0.13631849508918822
accuracy 0.794
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 19% | 75% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 19% | 35% |
Tensor: GPU pinned 32 × 3 × 3 × 3
Tensor: GPU pinned 32
Tensor: GPU pinned 32
Tensor: GPU pinned 32 × 1 × 3 × 3
Tensor: GPU pinned 32
Tensor: GPU pinned 32
Tensor: GPU pinned 16 × 32 × 1 × 1
Tensor: GPU pinned 16
Tensor: GPU pinned 1

FN,█▅▃▁▁
FN_test,▁▅█▄▃
FP,█▄▂▂▁
FP_test,▁█▁▁▇
TN,▁▄▆██
TN_test,█▄▁▅▆
TP,▁▅▇▇█
TP_test,█▁██▂
accuracy_test,█▁▂▆▄
accuracy_train,▁▅▇██
epoc,▁▃▅▆█


...

FN,█▁▁▁▁
FN_test,█▁▁▃▃
FP,█▁▁▁▁
FP_test,█▆▄▁▇
TN,▁████
TN_test,▁██▆▆
TP,▁████
TP_test,▁▃▅█▂
accuracy_test,▁▇█▇▅
accuracy_train,▁████
epoc,▁▃▅▆█


wandb: Sweep Agent: Waiting for job.
wandb: Job received.
wandb: Agent Starting Run: 3usa5pu3 with config:
wandb: 	batch_size: 16
wandb: 	epochs: 5
wandb: 	lr: 0.00011130066903909835
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Total size: 0


start Epoch  0
Client ID  , epoch  0 :  0.1754891948619388
accuracy 0.989375
test_loss 0.2654038741445494
accuracy 0.8263888888888888
start Epoch  1
Client ID  , epoch  1 :  0.15630500825591298
accuracy 0.99125
test_loss 0.28349131637064384
accuracy 0.8323412698412699
start Epoch  2
Client ID  , epoch  2 :  0.21439601841277067
accuracy 0.987375
test_loss 0.31982895293021146
accuracy 0.8174603174603174
start Epoch  3
Client ID  , epoch  3 :  0.3118997976787341
accuracy 0.983125
test_loss 0.2706742842519094
accuracy 0.7946428571428571
start Epoch  4
Client ID  , epoch  4 :  0.20141826874714752
accuracy 0.98875
test_loss 0.27285825541900793
accuracy 0.814484126984127
Initial GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 11% | 97% |
GPU Usage after emptying the cache
| ID | GPU | MEM |
------------------
|  0 | 11% | 19% |
Tensor: GPU pinned 32 × 3 × 3 × 3
Tensor: GPU pinned 32
Tensor: GPU pinned 32
Tensor: GPU pinned 32 × 1 × 3 × 3
Tensor: GPU pinned 32
Tensor: GPU pinned 32
Tens

FN,▂▁▃█▃
FN_test,▂▁▂█▅
FP,▃▁▄█▃
FP_test,▁▁█▇▃
TN,▇█▆▁▆
TN_test,▇█▇▁▄
TP,▆█▅▁▆
TP_test,██▁▂▆
accuracy_test,▇█▆▁▄
accuracy_train,▇█▅▁▆
epoc,▁▃▅▆█


# Clear Memory GPU

In [9]:
def pretty_size(size):
	"""Pretty prints a torch.Size object"""
	assert(isinstance(size, torch.Size))
	return " × ".join(map(str, size))

def dump_tensors(gpu_only=True):
	"""Prints a list of the Tensors being tracked by the garbage collector."""
	import gc
	total_size = 0
	for obj in gc.get_objects():
		try:
			if torch.is_tensor(obj):
				if not gpu_only or obj.is_cuda:
					print("%s:%s%s %s" % (type(obj).__name__, 
										  " GPU" if obj.is_cuda else "",
										  " pinned" if obj.is_pinned else "",
										  pretty_size(obj.size())))
					total_size += obj.numel()
			elif hasattr(obj, "data") and torch.is_tensor(obj.data):
				if not gpu_only or obj.is_cuda:
					print("%s → %s:%s%s%s%s %s" % (type(obj).__name__, 
												   type(obj.data).__name__, 
												   " GPU" if obj.is_cuda else "",
												   " pinned" if obj.data.is_pinned else "",
												   " grad" if obj.requires_grad else "", 
												   " volatile" if obj.volatile else "",
												   pretty_size(obj.data.size())))
					total_size += obj.data.numel()
		except Exception as e:
			pass        
	print("Total size:", total_size)
dump_tensors()


Total size: 0


In [10]:
cf_matrix=0

def confusion_matrix(target,pred):
    for i in range(0,target.shape[0]):
        cf_matrix+=confusion_matrix(target[i], pred[i])
def f1_score(target,pred):
    for i in range(0,target.shape[0]):
       print(accuracy_score(target[i], pred[i]))
f1_score(a,b)

NameError: name 'a' is not defined